## This is the script for faiss similarity search for news discovery ##

In [1]:
#install faiss
!pip install faiss-cpu --no-cache

In [ ]:
#install sentence transformers
! pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 72.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 87.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.2 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=ffaeef673618be025418c433e8a57f7d9da56a8b5b1def69ff3b47fa20c113eb
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
import numpy as np
import faiss
import requests
from io import StringIO
import pandas as pd

# Dataset Initialization

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_json("/content/drive/MyDrive/Capstone-esya.ai/DATA/summarized_df100.json")
df

,article_url,section,text,title,website,word_count,summary
0,https://edition.cnn.com/2019/01/20/us/mlk-lega...,fact-check-politics,They are the of the civil rights movement: t...,A new Supreme Court is poised to take a chunk ...,edition.cnn.com,144,Racial entitlements are rights that apply to m...
1,https://edition.cnn.com/2018/11/05/us/angry-wh...,fact-check-politics,"""We're not generating enough angry white guys ...",When the dreaded 'other' is an angry white man,edition.cnn.com,188,Read this article and summarize it in 50 to 35...
2,https://edition.cnn.com/2020/08/08/us/federal-...,fact-check-politics,"Russ soon got another call: James, a 65-year-o...",Inside the federal prison where three out of e...,edition.cnn.com,230,Viral Hemorrhagic Fever (VHF) due to a novel-C...
3,https://edition.cnn.com/2020/06/23/us/protests...,fact-check-politics,"A Fort Lauderdale, Florida, policeman, facing ...",Want to reform the police? Hire more women,edition.cnn.com,119,The Fort Lauderdale police chief has suspended...
4,https://edition.cnn.com/2018/10/16/us/eugenics...,fact-check-politics,But many people were repelled by their prese...,When Americans tried to breed a better race: H...,edition.cnn.com,121,An editorial that includes analysis and policy...
...,...,...,...,...,...,...,...
95,https://edition.cnn.com/2020/04/01/politics/te...,fact-check-politics,New Jersey-based Quest Diagnostics had about 1...,"Documents show backlog of 160,000 coronavirus ...",edition.cnn.com,151,This article is from the Independent Journal R...
96,https://www.thequint.com/news/education/cbse-p...,Education,The Central Board of Secondary Education has s...,CBSE Practicals: Who Will Conduct? What Are th...,www.thequint.com,217,Body:\nCBSE has said that practical examinatio...
97,https://edition.cnn.com/2020/10/22/politics/vo...,fact-check-politics,Secretaries of state or election boards in 29 ...,States grapple with mask rules at polls to avo...,edition.cnn.com,167,Maine Secretary of State's office slammed as u...
98,https://edition.cnn.com/2020/01/17/politics/ev...,fact-check-politics,"Yang, the wife of Democratic presidential cand...",Evelyn Yang was assured justice. Here's how th...,edition.cnn.com,176,This article is very interesting because it go...


We will take all samples from `text` and build sentence embeddings for each - which we can then store in FAISS.

In [ ]:
#select only text for futher embedding
sentences = df['text'].tolist()
sentences[:5]

['They are the   of the civil rights movement: the 1964 Civil Rights Act, the 1965 Voting Rights Act and the Fair Housing Act of 1968. A new conservative bloc on the Supreme Court though may soon treat them as something else: outdated "racial entitlements" that need to be put back in their place.  That\'s the dreaded   some experts envision for these landmark laws now that Justice   has joined the Supreme Court. They warn that, for the first time,  the high court has five firmly conservative judges who were   to dismantle the legal legacy of these laws, which have stood for 50 years.  "They will chip, chip away at these laws until there is nothing left," says Carol Anderson, author of  "One Person, No Vote: How Voter Suppression Is Destroying Our Democracy." ',
 '"We\'re not generating enough angry white guys to stay in business for the long term,"  , while attending the 2012 Republican National Convention. No one is making that complaint anymore. Lately, the news seems filled with gri

In [ ]:
len(set(sentences))

100

Tokenized the words

In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('bert-base-nli-mean-tokens')

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

(100, 768)

We can save/load from file in the case of needing to reload the notebook for any reason later.

In [ ]:
sentence_embeddings.shape[0]

100

We setup our FAISS database dimensionality (number of dimensions per vector) based on these vectors.

In [ ]:
d = sentence_embeddings.shape[1]
d

768

# Flat L2 Index

We initialize the flat L2 distance index `IndexFlatL2`, all we need is the specify the vector dimensionality - which in this case is `d == 768` (to align with the sentence-BERT model output embeddings of size `768`).

In [ ]:
index = faiss.IndexFlatL2(d)

In [ ]:
index.is_trained

True

Okay so once we're happy that our index is prepared, we then add new vectors using the `add` method.

In [ ]:
index.add(sentence_embeddings)

In [ ]:
index.ntotal

100

Then search given a query `xq` and number of nearest neigbors to return `k`.

In [ ]:
k = 5
xq = model.encode(["covid situation"])

In [ ]:
%%time
D, I = index.search(xq, k)  # search
print(I)  # k-nearest neigbors of the query vector | nprobe == 1: 6495 26392 61709 49932 | nprobe == 10: 36245  6495 57489  8705

[[72 96 50  0  4]]
CPU times: user 790 µs, sys: 0 ns, total: 790 µs
Wall time: 650 µs


Here we're returning text of the news article, which returns:

In [ ]:
[f'{i}: {sentences[i]}' for i in I[0]]

['72: In a letter from the House Committee on Energy and Commerce, US Reps. Frank Pallone Jr. and Jan Schakowsky told Amazon CEO Jeff Bezos that CNN\'s   on AmazonBasics products had revealed "alarming product safety issues." "Amazon\'s oversight of its own products has been grossly inadequate," the letter from the two Democrats stated. "Despite starkly worded reviews and even photographs posted by consumers warning of the grave safety dangers experienced while using AmazonBasics products, it appears that Amazon has turned a blind eye to these problems, prioritizing sales at the expense of safety."  AmazonBasics is one of the retailer\'s popular private label lines, and CNN\'s investigation found that dozens of AmazonBasics electronics remained for sale despite customers reporting the products had melted, exploded or burst into flames.  At least 1,500 reviews on the Amazon website, covering more than 70 items, described safety risks. Within those reviews, many consumers explicitly call

In [ ]:
# Interactive search loop
while True:
    # Request user input for a query
    query_text = input("What do you want to search (or 'q' to quit): ")

    # Check if the user wants to quit
    if query_text.lower() == 'q':
        print("Exiting the search - thank you")
        break


    # Perform similarity search
    k = 5  # Number of nearest neighbors to retrieve
    xq = model.encode([query_text])
    distances, indices = index.search(xq, k)

    # Get similar texts
    results = df.iloc[indices[0]]

    # Display the similar texts
    print("\nSimilar Texts:")
    print(results)

What do you want to search (or 'q' to quit): retail

Similar Texts:
                                          article_url              section  \
72  https://edition.cnn.com/2020/10/07/business/am...  fact-check-politics   
96  https://www.thequint.com/news/education/cbse-p...            Education   
46  https://edition.cnn.com/2016/02/26/health/texa...  fact-check-politics   
84  https://edition.cnn.com/2015/03/18/business/ma...  fact-check-politics   
14  https://edition.cnn.com/2019/02/01/us/howard-t...  fact-check-politics   

                                                 text  \
72  In a letter from the House Committee on Energy...   
96  The Central Board of Secondary Education has s...   
46  One needs a bus from Texarkana to Shreveport, ...   
84  The country, located some 500 miles east of Ma...   
14   "Don't ask what the world needs, ask what mak...   

                                                title           website  \
72  Lawmakers say Amazon has 'turned a blind 

KeyboardInterrupt: ignored

In [ ]:
!pip install pyngrok
!pip install flask-ngrok
!ngrok authtoken '2XV2EPojlDHxP2tE06qohqFz08F_6Ao4LA9NCxTnpshhvCHMp'

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
app = Flask(__name__, template_folder='/content/drive/MyDrive')
run_with_ngrok(app)

@app.route("/")
def home():
    return render_template("index.html")

@app.route('/search', methods=['POST'])
def search():
    query = request.form['query']

    # Perform similarity search
    k = 5  # Number of nearest neighbors to retrieve
    xq = model.encode([query])
    distances, indices = index.search(xq, k)

    # Get similar texts
    results = df.iloc[indices[0]]

    # Display the similar texts
    print("\nSimilar Texts:")
    print(results)

    return render_template('results.html', tables=[results.to_html(classes='data')], titles=results.columns.values)

app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://19d3-104-154-16-174.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [06/Nov/2023 18:15:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Nov/2023 18:15:04] "GET /favicon.ico HTTP/1.1" 404 -



Similar Texts:
                                          article_url              section  \
72  https://edition.cnn.com/2020/10/07/business/am...  fact-check-politics   
96  https://www.thequint.com/news/education/cbse-p...            Education   
46  https://edition.cnn.com/2016/02/26/health/texa...  fact-check-politics   
84  https://edition.cnn.com/2015/03/18/business/ma...  fact-check-politics   
14  https://edition.cnn.com/2019/02/01/us/howard-t...  fact-check-politics   

                                                 text  \
72  In a letter from the House Committee on Energy...   
96  The Central Board of Secondary Education has s...   
46  One needs a bus from Texarkana to Shreveport, ...   
84  The country, located some 500 miles east of Ma...   
14   "Don't ask what the world needs, ask what mak...   

                                                title           website  \
72  Lawmakers say Amazon has 'turned a blind eye' ...   edition.cnn.com   
96  CBSE Practicals: W

INFO:werkzeug:127.0.0.1 - - [06/Nov/2023 18:24:17] "POST /search HTTP/1.1" 200 -
